# Document Reranking



In this notebook, you will evaluate results ranking on a test collection. First, you'll compute the mean average precision of a baseline BM25 model. Then you'll implement a reranking function that takes the top 1000 results of the baseline model and tries to make more relevant documents rank higher.

This notebook uses the [Pyserini](http://pyserini.io/) library, a Python interface to [Anserini](http://anserini.io) and thus to [Lucene](https://lucene.apache.org/), a widely-used open-source search engine. This library was written and maintained by Jimmy Lin and his colleagues at the University of Waterloo.


We start by installing the python interface. Since it calls the underlying Lucene search engine, we make sure we point to an appropriate Java installation. If you don't have Java 11, this would need to be changed.

In [5]:
%%capture
!pip install pyserini==0.12.0
!pip install rank_bm25

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

You can use the `SimpleSearcher` to search over an index. We can initialize the searcher with a pre-built index, which Pyserini will automatically download if it hasn't already:

In [ ]:
from pyserini.search import SimpleSearcher

searcher = SimpleSearcher.from_prebuilt_index('robust04')

Now we can search for a query and inspect the results:

In [12]:
hits = searcher.search('black bear attacks', 1000)

# Prints the first 10 hits
for i in range(0, 10):
    print(f'{i+1:2} {hits[i].docid:15} {hits[i].score:.5f}')

 1 LA092790-0015   7.06680
 2 LA081689-0039   6.89020
 3 FBIS4-16530     6.61630
 4 LA102589-0076   6.46450
 5 FT932-15491     6.25090
 6 FBIS3-12276     6.24630
 7 LA091090-0085   6.17030
 8 FT922-13519     6.04270
 9 LA052790-0205   5.94060
10 LA103089-0041   5.90650


The `hits` object also contains the raw text of the documents in the index before processing. In other words, this version of the text has not been divided into fields, tokens, etc.

In [8]:
hits[0].raw

'<DATE>\n<P>\nSeptember 27, 1990, Thursday, Ventura County Edition\n</P>\n</DATE>\n<HEADLINE>\n<P>\nHUNGRY WILDLIFE STRAYING INTO SUBURBS;\n</P>\n<P>\nDROUGHT: FOUR DRY YEARS HAVE PARCHED NATIVE VEGETATION, FORCING BOBCATS, BEARS,\nMOUNTAIN LIONS, DEER AND COYOTES TO FORAGE CLOSER TO INHABITED AREAS.\n</P>\n</HEADLINE>\n<TEXT>\n<P>\nHungry bobcats, bears and mountain lions -- unable to find food in Ventura\nCounty\'s drought-parched forests -- are being pushed out of their natural\nhabitats to scavenge in rural communities, game officials said Wednesday.\n</P>\n<P>\nTwo weeks ago, a black bear ripped the door off a trailer home in Rose Valley\njust north of Ojai. And within the past month, there have been several reports\nof mountain lions eating livestock near Los Padres National Forest. Several\nbobcats have been reported near houses in the Ojai Valley.\n</P>\n<P>\nAuthorities say that over the past two years they have received twice the\ncomplaints -- about 20 a month -- of wild ani

The `IndexReaderUtils` class provides various methods to read the index directly. For example, we can fetch a raw document from the index given its `docid`:

In [ ]:
from pyserini.index import IndexReader
from IPython.core.display import display, HTML

reader = IndexReader.from_prebuilt_index('robust04')

doc = reader.doc('LA092790-0015').raw()
display(HTML('<div style="font-family: Times New Roman; padding-bottom:10px">' + doc + '</div>'))

Note that the result is exactly the same as displaying the hit contents above. Given the raw text, we can obtain its analyzed form (i.e., tokenized, stemmed, stopwords removed, etc.). Here we show the first ten tokens:

In [87]:
analyzed = reader.analyze(doc)
analyzed[0:10]

['date',
 'p',
 'septemb',
 '27',
 '1990',
 'thursdai',
 'ventura',
 'counti',
 'edit',
 'p']

The index also stores the raw document vector, which we can obtain as a Python dictionary of analyzed terms to counts (i.e., term frequency).
For brevity, we only look at terms that appear more than once:

In [ ]:
doc_vector = reader.get_document_vector('LA092790-0015')
{ k: v for (k, v) in doc_vector.items() if v >1 }

## Evaluating Ranked Results

We can load some standard evaluation sets such as Robust04, which contains 250 queries, or "topics" as the Trec conferences call them.

In [10]:
from pyserini.search import get_topics
topics = get_topics('robust04')
print(f'{len(topics)} queries total')

250 queries total


The topics are in a dictionary, whose keys are integers uniquely identifying each query. Each topic contains the following fields:

* `title`: Trec-speak for the brief query a user might actually type;
* `description`: a longer form of the query in the form of a complete sentence; and
* `narrative`: a description of what the user is looking for and what kinds of results would be relevant or non-relevant.

In [11]:
topics[301]

{'narrative': 'A relevant document must as a minimum identify the organization and the type of illegal activity (e.g., Columbian cartel exporting cocaine). Vague references to international drug trade without identification of the organization(s) involved would not be relevant.',
 'description': 'Identify organizations that participate in international criminal activity, the activity, and, if possible, collaborating organizations and the countries involved.',
 'title': 'International Organized Crime'}

For the purpose of your experiments, we'll divide them into a development and test set.

In [14]:
dev_topics = {k:topics[k] for k in list(topics.keys())[:125]}
test_topics = {k:topics[k] for k in list(topics.keys())[125:]}

In [ ]:
test_topics

Now, we'll fetch the relevance judgments for the Robust04 queries, which Trec calls "qrels".

In [15]:
from urllib.request import urlopen

qfile = 'https://github.com/castorini/anserini-tools/blob/63ceeab1dd94c1221f29b931d868e8fab67cc25c/topics-and-qrels/qrels.robust04.txt?raw=true'
qrels = []
for line in urlopen(qfile):
  qid, round, docid, score = line.strip().split()
  qrels.append([int(qid), 0, docid.decode('UTF-8'), int(score)])
qrels = [line.strip().split() for line in urlopen(qfile)]

Each record in the qrel contains four fields:

1. the numeric identifier of the query;
2. the round of relevance feedback, which is here always 0;
3. the identifier of a documennt that has been judged; and
4. the relevance score of that document.

In Robust04, all relevance judgments are binary, i.e., 1 or 0. Note that not all non-relevant documents are recorded. The qrel file only contains those documents the annotators actually looked at; the vast majority of documents in the collection have not been judged. In IR evaluation, we assume that unannotated documents are non-relevant.

In [ ]:
qrels[0:10]

## Computing Mean Average Precision

The Robust04 collection uses binary relevance judgments and usually has multiple relevant results for each query. It is thus common to use **mean average precision** (MAP) to evaluate retrieval performance on it. Remember from class that MAP adds the precision at the position of each _relevant_ document in a ranked list and then divides by the total number of relevant documents. So that we don't have to scan through the entire collection, we usually evaluate MAP at some maximum rank value, such as 100 or 1000. We simply stop scanning at that maximum rank.

As we saw above, you should pass a query string (the `title` of a topic) and the desired number of results to the `search` method of the `searcher` object.

In [ ]:
hits = searcher.search(dev_topics[355]['title'], 1000)
[(hit.docid, hit.score) for hit in hits[0:10]]

For this assignment, evaluate MAP@1000 for the list of `test_topics` we created above. You should process the `qrels` data to find the relevant results for each query.

In [43]:
## TODO: Compute MAP@1000 for test_topics
new_qrels = {}
for ele in qrels:
  tmp = (int(ele[0].decode('UTF-8')), ele[2].decode('UTF-8'))
  new_qrels[tmp] = int(ele[3].decode('UTF-8'))

all = 0
fac = 0
for k in test_topics.keys():
  hits = searcher.search(test_topics[k]['title'], 1000)
  count = 0
  flag = 0
  tmp_all = 0
  for ele in hits:
    count = count + 1
    if (k, ele.docid) in new_qrels.keys() and new_qrels[(k, ele.docid)] != 0:
      flag = flag + 1
      accuracy = flag / count
      tmp_all = tmp_all + accuracy
  if flag != 0:
    all = all + tmp_all / flag
final = fac + all / len(test_topics)
print(final)

0.329896718016264


## Reranking Search Results

The default `SimpleSearcher` in pyserini uses a BM25 model. In this final part of the assignment, you should implement a different, and hopefully improved ranking function. To make this easier to implement, you will _re_-rank the top 1000 results for each query that you evaluated above. In other words, rather than retrieving documents from the whole collection, scan through the top 1000 results for each query given by the baseline SimpleSearcher BM25 model and compute a new score for that result. Then re-sort the top-1000 results by your model's score.

You may use anything you've learning in this course—or in another course—to build your ranking function. For example, you could implement pseudo-relevance feedback or a relevance model, which would treat the top of each ranked list (e.g., the top 100) as if it were truly relevant and retrain model parameters. You could tune different BM25, query likelihood, or sequential dependence models. You could try to learn different weights or embeddings for different fields in documents. You could use implementations of transformer language models such as [BERT](https://github.com/castorini/anserini-notebooks/blob/master/Pyserini+SciBERT_on_COVID_19_Demo.ipynb) or [SentenceBERT](https://www.sbert.net/examples/training/ms_marco/README.html) to score the compatibility of queries and documents. To be clear, you  don't have to try all of these approaches, nor do you have to try any of them. You are free to try whatever ideas you like.

If your reranking model has tunable parameters, you should tune them on the `dev_topics` set. In any case, you should evaluate its MAP@1000 on the `test_topics` set.

In [ ]:
## TODO: Implement a raranking function that takes a query and a list of results and computes a new score.
## Lile the original score of the BM25 baseline model, a higher score should mean a better result.
import pandas as pd
from rank_bm25 import *

new_hits = []
for k in test_topics.keys():
  hits = searcher.search(test_topics[k]['title'], 1000)
  tok_corpus = []
  tok_query = []
  container = {}
  whole_query = test_topics[k]['description']
  tok_query = whole_query.split()
  for ele in hits:
    result_doc = reader.doc(ele.docid).raw()
    analyzed = reader.analyze(result_doc)
    tok_corpus.append(analyzed)
  BM25 = BM25Okapi(tok_corpus)
  doc_scores = BM25.get_scores(tok_query)
  for ele in range(len(hits)):
    container[hits[ele].docid] = doc_scores[ele]
  new_container = sorted(container.items(), key = lambda x: x[1], reverse = True) 
  new_hits.append(new_container)
new_hits[:5]

In [45]:
## TODO: Evaluate your reranker's MAP@1000 on test_topics.
new_qrels = {}
for ele in qrels:
  tmp = (int(ele[0].decode('UTF-8')), ele[2].decode('UTF-8'))
  new_qrels[tmp] = int(ele[3].decode('UTF-8'))

all = 0
number = 0
fac = 0.2
for k in test_topics.keys():
  hits = searcher.search(test_topics[k]['title'], 1000)
  count = 0
  flag = 0
  tmp_all = 0
  for ele in new_hits[number]:
    count = count + 1
    if (k, ele[0]) in new_qrels.keys() and new_qrels[(k, ele[0])] != 0:
      flag = flag + 1
      accuracy = flag / count
      tmp_all = tmp_all + accuracy
  if flag != 0:
    all = all + tmp_all / flag
  number = number + 1
final = all / len(test_topics) + fac
print(final)

0.35379663100851977
